# Import Libraries

In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

# Startup Webdriver

In [2]:
driver = webdriver.Chrome("C:\Selenium drivers\chromedriver.exe")

In [3]:
url = "https://www.amazon.com"
driver.get(url)

### Create a function that will insert a search term using string formatting

In [4]:
def get_url(search_term):
    """generate a url from search term"""
    template = "https://www.amazon.com/s?k={}&crid=27E68Q3L17CGO&sprefix=data+%2Caps%2C420&ref=nb_sb_ss_ts-doa-p_2_5"
    search_term = search_term.replace( " ", "+")
    return template.format(search_term)

In [5]:
url = get_url("data analysis")
print(url)

https://www.amazon.com/s?k=data+analysis&crid=27E68Q3L17CGO&sprefix=data+%2Caps%2C420&ref=nb_sb_ss_ts-doa-p_2_5


In [6]:
driver.get(url)

## Extract page contents from the HTML in the background

### Step 1: create soup object

In [7]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [8]:
results = soup.find_all("div", {"data-component-type": "s-search-result" })

In [9]:
len(results)

48

### Step 2: Extract a single record

What I need
1. Book_name
2. url
3. Price
4. Rating

In [10]:
item = results[0]

In [11]:
atag = item.h2.a

In [12]:
book_name =atag.text.strip()

In [13]:
url = "https://www.amazon.com" + atag.get("href")

In [14]:
price_parent = item.find("span", "a-price")

In [16]:
price = price_parent.find("span", "a-offscreen").text

In [18]:
rating = item.i.text

### Step 3: Write a function that includes all other records

In [19]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    # book_name and url
    atag = item.h2.a
    book_name = atag.text.strip()
    url = "https://www.amazon.com" + atag.get("href")
    
    # price
    price_parent = item.find("span", "a-price")
    price = price_parent.find("span", "a-offscreen").text
    
       
    # rating 
    rating = item.i.text
        
    
    """I'll create a tuple that contains these items and i'll assign it to a result variable and I'll return the tuple as a result """
    result = (book_name, price, rating, url)
    
    return result

### Step 4

1. Create a records list that is empty that will contain all of our extracted records.
2. Use the results pattern above to collect all the results on the page to iterate through.
3. Check to see if the record list is empty or not.

In [20]:
records = []
results = soup.find_all("div", {"data-component-type": "s-search-result" })

for item in results:
        records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'find'

In [21]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    # book_name and url
    atag = item.h2.a
    book_name = atag.text.strip()
    url = "https://www.amazon.com" + atag.get("href")
    
    try:
        # price
        price_parent = item.find("span", "a-price")
        price = price_parent.find("span", "a-offscreen").text
    except AttributeError:
        return
    
    try:   
        # rating 
        rating = item.i.text
    except AttributeError:
        rating = " "
        
    
    """I'll create a tuple that contains these items and i'll assign it to a result variable and I'll return the tuple as a result """
    result = (book_name, price, rating, url)
    
    return result

In [22]:
records = []
results = soup.find_all("div", {"data-component-type": "s-search-result" })

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

#### Print the first result

In [26]:
records[0]

('Storytelling with Data: A Data Visualization Guide for Business Professionals',
 '$19.25',
 '4.6 out of 5 stars',
 'https://www.amazon.com/Storytelling-Data-Visualization-Business-Professionals/dp/1119002257/ref=sr_1_1?crid=27E68Q3L17CGO&keywords=data+analysis&qid=1639655944&sprefix=data+%2Caps%2C420&sr=8-1')

#### Print the prices

In [27]:
for row in records:
    print(row[1])

$19.25
$49.99
$14.00
$50.14
$55.24
$80.75
$11.92
$38.37
$35.89
$15.60
$37.80
$36.78
$23.95
$27.80
$33.48
$27.58
$19.47
$35.71
$44.94
$30.54
$40.49
$35.13
$22.99
$15.90
$30.32
$27.26
$29.95
$28.33
$39.99
$17.99
$56.72
$37.74
$115.00
$18.59
$18.99
$33.24
$34.99
$40.10
$53.62
$43.96
$40.00
$46.54
$17.97
$32.88


## Scrape Multiple Pages

In [28]:
def get_url(search_term):
    """generate a url from search term"""
    template = "https://www.amazon.com/s?k={}&crid=27E68Q3L17CGO&sprefix=data+%2Caps%2C420&ref=nb_sb_ss_ts-doa-p_2_5"
    search_term = search_term.replace( " ", "+")
    
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += "&page={}"
    
    return url

In [31]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    """generate a url from search term"""
    template = "https://www.amazon.com/s?k={}&crid=27E68Q3L17CGO&sprefix=data+%2Caps%2C420&ref=nb_sb_ss_ts-doa-p_2_5"
    search_term = search_term.replace( " ", "+")
    
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += "&page={}"
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    # book_name and url
    atag = item.h2.a
    book_name = atag.text.strip()
    url = "https://www.amazon.com" + atag.get("href")
    
    try:
        # price
        price_parent = item.find("span", "a-price")
        price = price_parent.find("span", "a-offscreen").text
    except AttributeError:
        return
    
    try:   
        # rating 
        rating = item.i.text
    except AttributeError:
        rating = " "
        
    
    #I'll create a tuple that contains these items and i'll assign it to a result variable and I'll return the tuple as a result """
    result = (book_name, price, rating, url)
    
    return result

def main(search_term):
    #run main program"""
    #startup webdriver
    driver = webdriver.Chrome("C:\Selenium drivers\chromedriver.exe")
    
    #create the records list and set a url with our search term that we already passed to the main function"""
    records = []
    url = get_url(search_term)
    
     
    for page in range (1, 8):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = soup.find_all("div", {"data-component-type": "s-search-result" })
        print(page)
        
         # for each of the results, extract the records data and append it to the record's list
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    # save the data to a csv file
    with open("data_analyst.csv", "w", newline ="", encoding = "utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Book", "Price", "Rating", "Url"])
        writer.writerows(records)

In [32]:
main("data analysis")

1
2
3
4
5
6
7
